In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Use GPU if available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Clinical-T5-Large model and tokenizer

In [ ]:
model_name = "luqh/ClinicalT5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, from_flax = True)

# Example medical text

In [ ]:
medical_text = """
She does have asthma but doest not require daily medication for this and does not think it is flaring up.,
MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,
ALLERGIES: , She has no known medicine allergies.,
OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,
HEENT:  Her throat was mildly erythematous without exudate. 
Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  
TMs were clear.,
Neck:  Supple without adenopathy.,
Lungs:  Clear.,
ASSESSMENT:,  Allergic rhinitis.,PLAN:,1. 
"""

# Inference

In [ ]:
inputs = tokenizer.encode("summarize in 20 words: " + medical_text, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)